In [2]:
import pickle
import pandas as pd
import numpy as np
from scipy import stats
from collections import defaultdict
import random
import copy
import math
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
import time
import pulp
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx

# 1. Load input

Load episode set $E$ with the users that retweeted each original tweet in the trace. 

Each episode $E_{s}$ includes the users that retweeted s, ordered chronologically, as they appear in the trace. The first user in each episode is the user that originally tweeted the tweet, and is denoted by $r_{s}$. Subsequent users in $E_{s}$ are users that retweeted s, either directly from user $r_{s}$ or from another user that appears in Es before them.

In [6]:
lines = 500000
random.seed(10)
dataset = 'russian'

In [8]:
E = pickle.load(open("./Extracted/"+dataset+"/E"+ str(lines) + ".p", "rb"))

In [9]:
for s in E:
    E[s] = list(dict.fromkeys(E[s]))

Load the set of original tweets denoted by $S$. 

The set of original tweets is denoted by $S$, where $S$ = S is the total number of original tweets

In [10]:
S = pickle.load(open("./Extracted/"+dataset+"/S"+ str(lines) + ".p", "rb"))

Load $U$ set xith unique users

In [11]:
U = pickle.load(open("./Extracted/"+dataset+"/U"+ str(lines) + ".p", "rb"))
U = list(U)

Load $D$ dictionary

In [12]:
D = pickle.load(open("./Extracted/"+dataset+"/D"+ str(lines) + ".p", "rb"))

# 2. Find important quantities

- Number of unique users N
- Number of episodes / original tweets S
- Number of active pairs (i,j)

For every active pair we have to count the $\sigma_{ij}$'s

In [13]:
N = len(U)
print('Number of unique users N =', N)

Number of unique users N = 42011


In [14]:
print('Number of Episodes (original tweets) S =',len(E))

Number of Episodes (original tweets) S = 14781


# 3. Saito's Algorithm

In [15]:
def flatten(obj):
    if type(obj) == list:
        return [l for L in obj for l in L]
    if type(obj) == dict:
        return [l for i in obj for l in obj[i].values()]
    if type(obj) == defaultdict:
        return [l for i in obj for l in obj[i].values()]

In [16]:
# def saito(eps, D):
#         """ 
#         This function is the main algorithm for path inference with constraints. 

#         Parameters
#         ----------
#         rep : int 
#             Number of times to run the algorithm.
#         N : int
#             Number of users in the trace.
#         M : dict
#             Dictionary with M_{ij} values tha show how many times an ordered pair (i,j) appears in the trace.

#         Returns
#         ----------

#         """

#         iterat = 1
#         # ======================== INITIALIZE ========================
#         print('Initialize...')

#         # 0.1 -- Initialize k_{u,w}'s, count Splus, Sminus values 

#         k = defaultdict(dict)
#         Splus = defaultdict(dict)
#         Sminus = defaultdict(dict)

#         for s in D:
#                 for t in list(D[s]):
#                         if t!=0:
#                             for w in D[s][t]:
#                                 indx = list(D[s]).index(t)
#                                 previous_t = list(D[s])[indx-1]
#                                 for u in list(D[s][previous_t]):
#                                     k[u][w] = random.uniform(0,1)
#                                     if u in Splus and w in Splus[u]:
#                                         Splus[u][w].append(s)
#                                     else:
#                                         Splus[u][w]=[s]

#         for u in k:
#             for w in k[u]:
#                 for s in D:
#                     if u in flatten(list(D[s].values())) and s not in Splus[u][w]:
#                         if u in Sminus and w in Sminus[u]:
#                             Sminus[u][w].append(s)
#                         else:
#                             Sminus[u][w]=[s]
        

#         for u in k:
#             for w in k[u]:
#                 if w not in Sminus[u]:
#                     Sminus[u][w] = [] 

#         # ======================== START algorithm ========================

#         while True: # Repeat until convergence
            
#             # Step 1 ==== UPDATE VALUES ====

#             # 1.1 -- Update Pws dictionary
            
#             P = defaultdict(dict)

#             # update Pw
#             for s in D:
#                     for t in list(D[s]):
#                             if t!=0:
#                                 for w in D[s][t]:
#                                     indx = list(D[s]).index(t)
#                                     previous_t = list(D[s])[indx-1]
#                                     pr = 1 
#                                     for u in list(D[s][previous_t]):
#                                         pr*=(1-k[u][w])
#                                     P[w][s] = 1 - pr 
                                    
#             # update k-u-ws
#             for u in k:
#                 for w in k[u]:
#                     k[u][w] = (1/(len(Splus[u][w])+ len(Sminus[u][w]))) *sum(k[u][w]/P[w][s] for s in Splus[u][w])
            
            
#             # Step 2 ==== CHECK CONVERGENCE ====
            
#             # print('Check convergence...')

#             if iterat > 1:
#                 new_k = np.array(flatten(k))
#                 new_P = np.array(flatten(P))
                
#                 changek = np.linalg.norm(new_k - old_k)                  
#                 changeP = np.linalg.norm(new_P - old_P)  

#                 if changek < eps and changeP < eps: 
#                     print('-----')
#                     print('- Change k:', changek, 'Change P:', changeP)
                
#                     old_k = np.array(flatten(k))
#                     old_P = np.array(flatten(P))
#                     break
#                 else: 
#                     print('-----')
#                     print('- Change k:', changek, 'Change P:', changeP)#                     
                
#                     old_k = np.array(flatten(k))
#                     old_P = np.array(flatten(P))
                    
#                     iterat += 1
                    
#             if iterat == 1:
#                 flag = False
#                 old_k = np.array(flatten(k))
#                 old_P = np.array(flatten(P))
                
#                 iterat+=1
                
#         # ======================== END algorithm ========================
    
#         return iterat, k, P, Splus, Sminus

In [17]:
def saito(eps, D):
        """ 
        This function is the main algorithm for path inference with constraints. 

        Parameters
        ----------
        rep : int 
            Number of times to run the algorithm.
        N : int
            Number of users in the trace.
        M : dict
            Dictionary with M_{ij} values tha show how many times an ordered pair (i,j) appears in the trace.

        Returns
        ----------

        """

        iterat = 1
        # ======================== INITIALIZE ========================
        print('Initialize...')

        # 0.1 -- Initialize k_{u,w}'s, count Splus, Sminus values 

        k = defaultdict(dict)
        Splus = defaultdict(dict)
        Sminus = defaultdict(dict)
        print('Count splus, sminus..')
        print('Part 1..')
        for s in D:
                for t in list(D[s]):
                        if t!=0:
                            for w in D[s][t]:
                                indx = list(D[s]).index(t)
#                                 previous_t = list(D[s])[indx-1]
                                previous_t = list(D[s])[indx-1:indx]
                                for p_t in previous_t:
                                    for u in list(D[s][p_t]):
                                        k[u][w] = random.uniform(0,1)
                                        if u in Splus and w in Splus[u]:
                                            Splus[u][w].append(s)
                                        else:
                                            Splus[u][w]=[s]
        print('Part 2..')
        
        for s in D:
            for u in flatten(list(D[s].values())):
                for w in k[u]:
                    if s not in Splus[u][w]:
                        if u in Sminus and w in Sminus[u]:
                            Sminus[u][w].append(s)
                        else:
                            Sminus[u][w]=[s]                        
            
        
        
#         for u in k:
#             for w in k[u]:
#                 for s in D:
#                     if u in flatten(list(D[s].values())) and s not in Splus[u][w]:
#                         if u in Sminus and w in Sminus[u]:
#                             Sminus[u][w].append(s)
#                         else:
#                             Sminus[u][w]=[s]
        
        print('Part 3..')
        for u in k:
            for w in k[u]:
                if w not in Sminus[u]:
                    Sminus[u][w] = [] 
            

        # ======================== START algorithm ========================

        while True: # Repeat until convergence
            print('Begin algorithm..')
            # Step 1 ==== UPDATE VALUES ====

            # 1.1 -- Update Pws dictionary
            
            P = defaultdict(dict)

            # update Pw
            for s in D:
                    for t in list(D[s]):
                            if t!=0:
                                for w in D[s][t]:
                                    indx = list(D[s]).index(t)
                                    previous_t = list(D[s])[indx-1:indx]
#                                     previous_t = list(D[s])[:indx]
                                    pr = 1 
                                    for p_t in previous_t:
                                        for u in list(D[s][p_t]):
                                            pr*=(1-k[u][w])
                                    P[w][s] = 1 - pr 
                                    
            # update k-u-ws
            for u in k:
                for w in k[u]:
                    k[u][w] = (1/(len(Splus[u][w])+ len(Sminus[u][w]))) *sum(k[u][w]/P[w][s] for s in Splus[u][w])
            
            
            # Step 2 ==== CHECK CONVERGENCE ====
            
            # print('Check convergence...')

            if iterat > 1:
                new_k = np.array(flatten(k))
                new_P = np.array(flatten(P))
                
                changek = np.linalg.norm(new_k - old_k)                  
                changeP = np.linalg.norm(new_P - old_P)  

                if changek < eps and changeP < eps: 
                    print('-----')
                    print('- Change k:', changek, 'Change P:', changeP)
                
                    old_k = np.array(flatten(k))
                    old_P = np.array(flatten(P))
                    break
                else: 
                    print('-----')
                    print('- Change k:', changek, 'Change P:', changeP)#                     
                
                    old_k = np.array(flatten(k))
                    old_P = np.array(flatten(P))
                    
                    iterat += 1
                    
            if iterat == 1:
                flag = False
                old_k = np.array(flatten(k))
                old_P = np.array(flatten(P))
                
                iterat+=1
                
        # ======================== END algorithm ========================
    

        return iterat, k, P, Splus, Sminus

In [18]:
iterat, k_1, P, Splus, Sminus = saito(10**-3, D)

Initialize...
Count splus, sminus..
Part 1..
Part 2..
Part 3..
Begin algorithm..
Begin algorithm..
-----
- Change k: 5.157763332977815 Change P: 227.45416383520197
Begin algorithm..
-----
- Change k: 2.055412160415387 Change P: 5.122170019952855
Begin algorithm..
-----
- Change k: 1.1611448564900924 Change P: 1.547983263917167
Begin algorithm..
-----
- Change k: 0.8088148320295678 Change P: 0.7398042309769783
Begin algorithm..
-----
- Change k: 0.71418114694618 Change P: 0.3951717602512342
Begin algorithm..
-----
- Change k: 0.6120197615277477 Change P: 0.356441563171054
Begin algorithm..
-----
- Change k: 0.44974264392812024 Change P: 0.3208082530079135
Begin algorithm..
-----
- Change k: 0.25872665816900925 Change P: 0.25362432327732853
Begin algorithm..
-----
- Change k: 0.13999689201023385 Change P: 0.13792947661559504
Begin algorithm..
-----
- Change k: 0.09000421825964758 Change P: 0.049379074164949924
Begin algorithm..
-----
- Change k: 0.06577070210609134 Change P: 0.0215938821

-----
- Change k: 0.001952444766222934 Change P: 2.1416888489909087e-05
Begin algorithm..
-----
- Change k: 0.0019274065040698323 Change P: 2.0823221053481435e-05
Begin algorithm..
-----
- Change k: 0.0019029763157911114 Change P: 2.0254984834879818e-05
Begin algorithm..
-----
- Change k: 0.001879132874029316 Change P: 1.971083092691273e-05
Begin algorithm..
-----
- Change k: 0.0018558558092936967 Change P: 1.9189502460112578e-05
Begin algorithm..
-----
- Change k: 0.0018331256415665919 Change P: 1.8689826698385614e-05
Begin algorithm..
-----
- Change k: 0.0018109237185877614 Change P: 1.8210707793267674e-05
Begin algorithm..
-----
- Change k: 0.0017892321602433784 Change P: 1.7751120155144135e-05
Begin algorithm..
-----
- Change k: 0.0017680338085114987 Change P: 1.7310102394259066e-05
Begin algorithm..
-----
- Change k: 0.0017473121824441078 Change P: 1.6886751796632957e-05
Begin algorithm..
-----
- Change k: 0.001727051437699258 Change P: 1.6480219297051064e-05
Begin algorithm..
---

In [19]:
pickle.dump(k_1, open("./Extracted/"+dataset+"/results/k_saito_"+ str(lines) + ".p", "wb"))

In [17]:
# # what edges where drawn
# for tweet in E:
#         print('Episode:', tweet, E[tweet])
#         for i in E[tweet]:
#             for j in k_1[i]:
#                 if k_1[i][j] > 0.5 and j in E[tweet]:
#                     print('edge from', i, 'to', j, 'kij:', k_1[i][j])


Episode: 956216939172499456 [28859692, 209735575, 99659322, 239467085, 3044638328, 1351346624, 383935923, 431177479, 2404814880]
edge from 28859692 to 209735575 kij: 1.0
edge from 383935923 to 431177479 kij: 1.0
Episode: 956218141184577536 [4086524967, 1118449106]
Episode: 956218574967902208 [264912416, 40522512]
edge from 264912416 to 40522512 kij: 1.0
Episode: 956218666508521472 [2613120188, 856643726, 842298422505926656, 923858883285864448, 741911176213364736, 2420922070, 2516087875]
Episode: 956219412377407488 [386829998, 938362398196535296, 798799291, 318867335]
edge from 938362398196535296 to 798799291 kij: 1.0
edge from 798799291 to 318867335 kij: 1.0
Episode: 956219470908919808 [2513730044, 839741275066810368, 2369976362]
Episode: 956220249564700672 [2525561910, 433231620, 3108190078, 828245390071848960, 2436689696]
edge from 2525561910 to 433231620 kij: 1.0
Episode: 956220845629861888 [497135025, 1551531896, 2542991089, 2239497228, 1894796587, 741911176213364736, 7279110256521

edge from 532011549 to 870294579525844992 kij: 1.0
edge from 870294579525844992 to 2340848054 kij: 1.0
edge from 2340848054 to 956307160723419136 kij: 1.0
edge from 956307160723419136 to 955965418782511104 kij: 1.0
edge from 956307160723419136 to 955965751873204224 kij: 1.0
edge from 956299876299624448 to 956037606038540288 kij: 1.0
Episode: 956307359818747904 [773551912943779840, 109516993]
edge from 773551912943779840 to 109516993 kij: 1.0
Episode: 956308003258552320 [1954120872, 2479235838]
Episode: 956308280892063744 [956302686986653696, 711500594645749760]
edge from 956302686986653696 to 711500594645749760 kij: 1.0
Episode: 956308954551898112 [325779471, 527374829]
edge from 325779471 to 527374829 kij: 1.0
Episode: 956314605738975232 [4018307595, 2872089275, 728601000005804032, 2851567917]
edge from 4018307595 to 2872089275 kij: 1.0
Episode: 956319331733917696 [3064434064, 929179104192139264]
Episode: 956323684301582336 [200036850, 777718221730353152, 2223051344, 271407821, 247923

Episode: 956433550261260288 [369417512, 950430705351364608, 1136418756, 329105284, 531930031, 251661346, 3079347556, 2563398176]
edge from 369417512 to 950430705351364608 kij: 1.0
edge from 251661346 to 3079347556 kij: 1.0
edge from 3079347556 to 2563398176 kij: 1.0
Episode: 956433735230148608 [601946663, 948844603985326080, 800007046296993792]
edge from 948844603985326080 to 800007046296993792 kij: 1.0
Episode: 956433866306224128 [906087220708245504, 445393786, 2321897600, 890225883155443712, 1850792372, 2840393223, 445428719, 4128377992, 572555396, 2486259791, 350126539, 741735495097520128, 789477231374569472, 365354490]
edge from 890225883155443712 to 1850792372 kij: 1.0
Episode: 956433969117122560 [1958993382, 934067814561910784, 434668135, 939359392595574784, 3070059946, 1727406984, 2436689696, 475032714, 3563918596, 876331750380777472, 623995765, 817239620, 2269324536, 712968969905389568, 3735256708, 839627742]
edge from 2269324536 to 712968969905389568 kij: 1.0
Episode: 95643405

Episode: 956470441497055232 [2384091714, 2807118956]
Episode: 956470762264760320 [34200559, 293404489]
Episode: 956470927931461632 [2613120188, 2792560947, 878205503419158528, 1591846393, 716666671520415744, 572555396, 235920456, 853920507888107520, 872393807094050816, 445433035, 1698608305, 350126539, 2840393223, 750391573712699392, 1850792372]
Episode: 956471013386149888 [721493858996129792, 2479235838]
edge from 721493858996129792 to 2479235838 kij: 1.0
Episode: 956471261311459328 [3309099591, 2531231752, 421110732, 258015937, 257423694, 4635994217, 955872714757197824]
edge from 3309099591 to 2531231752 kij: 1.0
Episode: 956471357411246080 [484522929, 1457061193]
Episode: 956471362008371200 [3126133919, 2567223356, 376081637, 2367367311, 2848105953, 2668831447, 754332282304036864, 825385158807322624, 2377540410, 4410646635, 2565546990, 847163015900418048, 862552644493217792, 1369309567, 3069902578, 408847537, 827275053276368896, 910263226746703872, 393295969, 4837801228]
edge from 3

Episode: 956489605049659392 [1691765786, 1707447859]
edge from 1691765786 to 1707447859 kij: 1.0
Episode: 956490441897242624 [2321852454, 223034624, 929179104192139264]
edge from 223034624 to 929179104192139264 kij: 1.0
Episode: 956490610806059008 [954769012994576384, 287493868, 821480708313088000, 229595670, 712968969905389568, 1329128316]
edge from 954769012994576384 to 287493868 kij: 1.0
Episode: 956490646134673408 [2400073202, 821246941925412864, 294332076, 841313546248048640, 797815214171418624]
Episode: 956490684491583488 [2790104509, 3139355386]
Episode: 956490948502020096 [2794320208, 434668135]
Episode: 956491282871980032 [2939837158, 576918395, 895267107834929152]
Episode: 956491418905833472 [2939837158, 576918395]
Episode: 956491846372425728 [926451092967346176, 37218742, 43896120]
Episode: 956491889334784000 [494076761, 40807205, 3063567088, 936269254340493312, 819268341835661312, 873608385786261504, 1057025941, 333039375, 782965827054239744, 474392066, 2478769261, 31262552

...

edge from 369358783 to 447395352 kij: 1.0
edge from 325527297 to 3310655487 kij: 1.0
edge from 3304488244 to 722979679 kij: 1.0
edge from 1186064359 to 802779173928501248 kij: 1.0
edge from 120178218 to 845975941440507904 kij: 1.0
edge from 240047860 to 4203631186 kij: 1.0
edge from 4203631186 to 1449290017 kij: 1.0
edge from 3216936119 to 897719037148483584 kij: 1.0
edge from 256035357 to 3357447898 kij: 1.0
edge from 951143351339937792 to 815054131580825600 kij: 1.0
edge from 4870229014 to 1679038740 kij: 1.0
edge from 463906265 to 768129833796833280 kij: 1.0
edge from 788074175088951296 to 2489436282 kij: 1.0
edge from 2489436282 to 4918254700 kij: 1.0
edge from 4918254700 to 3401307003 kij: 1.0
edge from 3401307003 to 358118086 kij: 1.0
edge from 832911222 to 862849148 kij: 1.0
edge from 3245921465 to 2856436857 kij: 1.0
edge from 825942356 to 448091194 kij: 1.0
edge from 4868565653 to 535042590 kij: 1.0
Episode: 957542108738486272 [111975224, 4694911171]
edge from 111975224 to 469

edge from 593068050 to 535265672 kij: 1.0
edge from 535265672 to 3233094977 kij: 1.0
edge from 4220234939 to 259255929 kij: 1.0
edge from 4220234939 to 2367643048 kij: 1.0
edge from 847485635652755456 to 774325668008042496 kij: 1.0
edge from 774325668008042496 to 815932896770924544 kij: 1.0
edge from 815932896770924544 to 575353502 kij: 1.0
edge from 936196033 to 466560155 kij: 1.0
edge from 863763050171596800 to 736280719 kij: 1.0
edge from 736280719 to 422563204 kij: 1.0
edge from 2313783336 to 554370443 kij: 1.0
edge from 554370443 to 2900552636 kij: 1.0
edge from 2900552636 to 152303291 kij: 1.0
edge from 2924531681 to 2977012846 kij: 1.0
edge from 2977012846 to 2570385957 kij: 1.0
edge from 27659971 to 574207293 kij: 1.0
edge from 574207293 to 845975941440507904 kij: 1.0
edge from 2235037259 to 1679038740 kij: 1.0
edge from 514024935 to 367632856 kij: 1.0
edge from 2279092135 to 2156859164 kij: 1.0
edge from 406366931 to 104465352 kij: 1.0
edge from 437327932 to 1583565216 kij: 1.

edge from 931614430189838336 to 408285035 kij: 1.0
edge from 408285035 to 946642309256499200 kij: 1.0
edge from 946642309256499200 to 939231606887407616 kij: 1.0
Episode: 957569596256636928 [293552956, 841030324456980480]
Episode: 957569714183704576 [584063322, 115612698, 708987517752745984, 2479235838, 362042036, 305659302, 811164414980329472]
edge from 362042036 to 305659302 kij: 1.0
Episode: 957569893741793280 [2836965627, 3930300933]
edge from 2836965627 to 3930300933 kij: 1.0
Episode: 957570225548873728 [2364050814, 332890763]
Episode: 957570228724092928 [2567223356, 815054131580825600, 885466323408617472]
Episode: 957570388761956352 [1111234566, 198446323, 586851660, 484151195, 1071123338, 525122063, 384999027, 2801843863, 943792919793856512, 755324544345378816, 2666780155, 377524256, 32898319, 879262598008627200, 3126255262, 290532768, 4382900482, 226139753, 774971430685597696, 2570468423, 112515691, 741911176213364736, 4177890263, 845961410043691008, 1706630467, 824501857, 2396

edge from 146808904 to 869437258767761408 kij: 1.0
Episode: 957581429420056576 [906740454, 883777999094648832]
edge from 906740454 to 883777999094648832 kij: 1.0
Episode: 957581510944743424 [3171065087, 3172047629, 439361064]
Episode: 957581706260819968 [2653528316, 1385750406, 2846562025]
Episode: 957581991175581696 [871628460057387008, 384999027]
Episode: 957582006455558144 [3953707215, 848453158129029120]
edge from 3953707215 to 848453158129029120 kij: 1.0
Episode: 957582022884675584 [783344088292663296, 1012596764, 843886695577800704, 742574298834083840, 422563204, 1085398099, 924344537593663488, 794433469535440896, 274367522, 2523059888, 852168903610511360]
Episode: 957582396366512128 [809488456967659520, 3172047629, 1424624701, 353868037, 851362060541841408, 376336190, 2949754366, 436083162]
edge from 353868037 to 851362060541841408 kij: 1.0
Episode: 957582583038140416 [512814104, 424285380, 358118086, 360793850, 2318518996, 4096285402, 732328172]
edge from 2318518996 to 40962854

edge from 342339853 to 534216009 kij: 1.0
Episode: 957598202076811264 [242414346, 716666671520415744, 797198903791747072, 1893791208, 753095426, 3132829052, 887382326262583296, 3330107775, 925093232647852032]
Episode: 957598370289381376 [3200003201, 767270527, 907959070010826752, 3892113987, 3930304774, 412937704, 2895695397, 492273879, 871628460057387008, 4744286789, 825554648861966336, 3242553260, 2851179723, 590599726, 351466872, 744952096898846720, 1384105404, 742982492, 510269988, 616549958, 933308616785973248, 900049577604247552, 884784313602068480, 3044695103, 2312202326, 3391712099, 718882676791492608, 3205648288, 706129073391665152, 2311982917, 394992914, 707447467, 4461862702, 2750356590, 876790072728748032, 472262799, 4625273146]
edge from 2851179723 to 590599726 kij: 1.0
edge from 744952096898846720 to 1384105404 kij: 1.0
edge from 3044695103 to 2312202326 kij: 1.0
edge from 3391712099 to 718882676791492608 kij: 1.0
edge from 3205648288 to 706129073391665152 kij: 1.0
edge f